In [41]:
from pymongo import MongoClient

# Assuming youre running mongod on 'localhost' with port 27017
client = MongoClient('localhost', 27017)

# We can also specify the host and port explicitly, as follows:
#client = MongoClient('mongodb://localhost:27017/')

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [42]:
client.database_names()

[u'donorschoose', u'local', u'twtr01_db']

In [44]:
db = client.test_database
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'test_database')

In [45]:
db = client['test-database']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'test-database')

In [48]:
collection = db.test_collection
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'test-database'), u'test_collection')

In [49]:
collection = db['test-collection']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'test-database'), u'test-collection')

In [50]:
import datetime
post = {"author": "Mike",
"text": "My first blog post!",
"tags": ["mongodb", "python", "pymongo"],
"date": datetime.datetime.utcnow()}

In [51]:
# Inserting a Document
# To insert a document into a collection we can use the insert_one() method:

posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id 

ObjectId('5ab0413d41bce534fcfdca02')

In [53]:
# Getting a Single Document With find_one()
import pprint
pprint.pprint(posts.find_one())

{u'_id': ObjectId('5ab0413d41bce534fcfdca02'),
 u'author': u'Mike',
 u'date': datetime.datetime(2018, 3, 19, 23, 1, 8, 510000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}


In [54]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{u'_id': ObjectId('5ab0413d41bce534fcfdca02'),
 u'author': u'Mike',
 u'date': datetime.datetime(2018, 3, 19, 23, 1, 8, 510000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}


In [55]:
# Querying By ObjectId
post_id

ObjectId('5ab0413d41bce534fcfdca02')

In [56]:
pprint.pprint(posts.find_one({"_id": post_id}))

{u'_id': ObjectId('5ab0413d41bce534fcfdca02'),
 u'author': u'Mike',
 u'date': datetime.datetime(2018, 3, 19, 23, 1, 8, 510000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}


In [57]:
post_id_as_str = str(post_id)
posts.find_one({"_id": post_id_as_str}) # No result

In [ ]:
'''
A common task in web applications is to get an ObjectId from the request URL and find the matching document. 
It’s necessary in this case to convert the ObjectId from a string before passing it to find_one:
'''
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

In [58]:
#Bulk Inserts

new_posts = [{"author": "Mike",
                "text": "Another post!",
                "tags": ["bulk", "insert"],
                "date": datetime.datetime(2009, 11, 12, 11, 14)},
                 {"author": "Eliot",
                "title": "MongoDB is fun",
                "text": "and pretty easy too!",
                "date": datetime.datetime(2009, 11, 10, 10, 45)}]

result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('5ab0420941bce534fcfdca03'), ObjectId('5ab0420941bce534fcfdca04')]

In [59]:
# Querying for More Than One Document
for post in posts.find():
    pprint.pprint(post)

{u'_id': ObjectId('5ab0413d41bce534fcfdca02'),
 u'author': u'Mike',
 u'date': datetime.datetime(2018, 3, 19, 23, 1, 8, 510000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}
{u'_id': ObjectId('5ab0420941bce534fcfdca03'),
 u'author': u'Mike',
 u'date': datetime.datetime(2009, 11, 12, 11, 14),
 u'tags': [u'bulk', u'insert'],
 u'text': u'Another post!'}
{u'_id': ObjectId('5ab0420941bce534fcfdca04'),
 u'author': u'Eliot',
 u'date': datetime.datetime(2009, 11, 10, 10, 45),
 u'text': u'and pretty easy too!',
 u'title': u'MongoDB is fun'}


In [60]:
for post in posts.find({"author": "Mike"}):
    pprint.pprint(post)

{u'_id': ObjectId('5ab0413d41bce534fcfdca02'),
 u'author': u'Mike',
 u'date': datetime.datetime(2018, 3, 19, 23, 1, 8, 510000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}
{u'_id': ObjectId('5ab0420941bce534fcfdca03'),
 u'author': u'Mike',
 u'date': datetime.datetime(2009, 11, 12, 11, 14),
 u'tags': [u'bulk', u'insert'],
 u'text': u'Another post!'}


In [61]:
# counting
# If we just want to know how many documents match a query we can perform a count() operation instead of a full query. 
# We can get a count of all of the documents in a collection:
posts.count()

3

In [62]:
# or just of those documents that match a specific query
posts.find({"author": "Mike"}).count()

2

In [63]:
# Range Queries
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{u'_id': ObjectId('5ab0420941bce534fcfdca04'),
 u'author': u'Eliot',
 u'date': datetime.datetime(2009, 11, 10, 10, 45),
 u'text': u'and pretty easy too!',
 u'title': u'MongoDB is fun'}
{u'_id': ObjectId('5ab0420941bce534fcfdca03'),
 u'author': u'Mike',
 u'date': datetime.datetime(2009, 11, 12, 11, 14),
 u'tags': [u'bulk', u'insert'],
 u'text': u'Another post!'}


In [64]:
# Index
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)], unique=True)
sorted(list(db.profiles.index_information()))

[u'_id_', u'user_id_1']

In [65]:
user_profiles = [{'user_id': 211, 'name': 'Luke'}, {'user_id': 212, 'name': 'Ziltoid'}]
result = db.profiles.insert_many(user_profiles)
result

In [66]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}
result = db.profiles.insert_one(new_profile)  # This is fine.
result = db.profiles.insert_one(duplicate_profile)
result

DuplicateKeyError: E11000 duplicate key error collection: test-database.profiles index: user_id_1 dup key: { : 212 }

In [67]:
# Alternative

In [6]:
import pymongo
import json

client = pymongo.MongoClient("localhost", 27017, maxPoolSize=50)
d = dict((db, [collection for collection in client[db].collection_names()])
         for db in client.database_names())
print json.dumps(d)

{"local": ["startup_log"], "twtr01_db": ["twtr01_coll"], "donorschoose": ["projects"]}


In [9]:
d['twtr01_db']

[u'twtr01_coll']

In [40]:
from pprint import pprint

cursor = collection.find({})
for document in cursor: 
    pprint(document)